In [6]:
import os
import random
import pkg_resources

In [7]:
expression_dir = "../Matrix-Chain-4/variants-linnea/experiments/100_100_10_100_100/"

In [8]:
runner_file = os.path.join(expression_dir,"runner.jl")

In [9]:
competing_algs = ['algorithm1', 'algorithm0']

In [10]:
variants_path = [os.path.join(expression_dir, "experiments", "{}.jl".format(alg)) for alg in competing_algs]

In [11]:
variants_path

['../Matrix-Chain-4/variants-linnea/experiments/100_100_10_100_100/experiments/algorithm1.jl',
 '../Matrix-Chain-4/variants-linnea/experiments/100_100_10_100_100/experiments/algorithm0.jl']

In [9]:
for var in variants_path:
    print(os.path.exists(var))

True
True


In [10]:
offset = '   '

In [17]:
runner_template = offset+'ret,times = {alg}(map(MatrixGenerator.unwrap, map(copy, matrices))...)\n'
runner_template += offset+'write_{alg}_to_eventlog(io, "{alg}_{rep}", times)\n\n'

In [12]:
R = 3
measurements_instance_set = []
for variant in competing_algs:
    measurements_instance_set = measurements_instance_set + [(i,variant) for i in range(R)]

In [13]:
measurements_instance_set

[(0, 'algorithm1'),
 (1, 'algorithm1'),
 (2, 'algorithm1'),
 (0, 'algorithm0'),
 (1, 'algorithm0'),
 (2, 'algorithm0')]

In [15]:
random.shuffle(measurements_instance_set)

In [16]:
measurements_instance_set

[(0, 'algorithm0'),
 (0, 'algorithm1'),
 (2, 'algorithm0'),
 (2, 'algorithm1'),
 (1, 'algorithm0'),
 (1, 'algorithm1')]

In [18]:
runner_code = ""
for measurement in measurements_instance_set:
    runner_code += runner_template.format(alg=measurement[1], rep=measurement[0])
    

In [20]:
print(runner_code)

   ret,times = algorithm0(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm0_to_eventlog(io, "algorithm0_0", times)

   ret,times = algorithm1(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm1_to_eventlog(io, "algorithm1_0", times)

   ret,times = algorithm0(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm0_to_eventlog(io, "algorithm0_2", times)

   ret,times = algorithm1(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm1_to_eventlog(io, "algorithm1_2", times)

   ret,times = algorithm0(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm0_to_eventlog(io, "algorithm0_1", times)

   ret,times = algorithm1(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm1_to_eventlog(io, "algorithm1_1", times)




In [50]:
def generate_runner_competing_code(competing_vars, reps, expression_dir=""):
    
    variants_path = [os.path.join(expression_dir, 
                                  "experiments", "{}.jl".format(alg)) for alg in competing_vars]
    for var_path in variants_path:
        if not os.path.exists(var_path):
            return -1
    
    variants_includes = ""
    measurements_instance_set = []
    for var in competing_vars:
        variants_includes += 'include("experiments/{}.jl")\n'.format(var)
        measurements_instance_set = measurements_instance_set + [(i,var) for i in range(reps)]
    random.shuffle(measurements_instance_set)
    
    
    runner_template = offset+'ret,times = {alg}(map(MatrixGenerator.unwrap, map(copy, matrices))...)\n'
    runner_template += offset+'write_{alg}_to_eventlog(io, "{alg}_{rep}", times)\n\n'
    runner_code = ""
    for measurement in measurements_instance_set:
        runner_code += runner_template.format(alg=measurement[1], rep=measurement[0])
        
    runner_path = os.path.join(expression_dir,"run_times_competing.txt")
    inject = {
        'variants_includes': variants_includes,
        'runner_code': runner_code,
        'runner_path': os.path.abspath(runner_path)
    }
    
    
    #template_str = pkg_resources.resource_string(__name__, "templates/runner.jl").decode("UTF-8")
    template_str = open("../utils/templates/runner.jl", "r").read()
    print(template_str.format(**inject))
    
#     runner_file = os.path.join(expression_dir,"runner_competing.jl")
#     with open(runner_file, "wt", encoding='utf-8') as output_file:
#         output_file.write(template_str.format(**inject))
    

    

In [51]:
generate_runner_competing_code(competing_algs,3,expression_dir)

using MatrixGenerator

include("experiments/algorithm1.jl")
include("experiments/algorithm0.jl")

include("operand_generator.jl")

function main()

    matrices = operand_generator()

    io = open("/Users/aravind/exercise/PyCharmProjects/performance-analyzer/Experiments2/Matrix-Chain-4/variants-linnea/experiments/100_100_10_100_100/run_times_competing.txt","w")
    write(io, "case:concept:name;concept:name;concept:flops;concept:operation;concept:kernel;timestamp:start;timestamp:end\n")

   ret,times = algorithm0(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm0_to_eventlog(io, "algorithm0_0", times)

   ret,times = algorithm0(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm0_to_eventlog(io, "algorithm0_1", times)

   ret,times = algorithm1(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_algorithm1_to_eventlog(io, "algorithm1_0", times)

   ret,times = algorithm0(map(MatrixGenerator.unwrap, map(copy, matrices))...)
   write_al

In [41]:
data = open("../utils/templates/runner.jl", "r").read()

In [32]:
print(data)

using MatrixGenerator

{variants_includes}
include("operand_generator.jl")

function main()

    matrices = operand_generator()

    io = open("{runner_path}","w")
    write(io, "case:concept:name;concept:name;concept:flops;concept:operation;concept:kernel;timestamp:start;timestamp:end\n")

{runner_code}


end

main()


In [12]:
import sys
sys.path.append("../utils/")

In [13]:
from generate_linnea_experiment_code import generate_runner_competing_code

In [14]:
generate_runner_competing_code(competing_algs,5,expression_dir)